In [3]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import tensorflow.keras.layers as layers

In [4]:
#Load data
data = pd.read_csv('Medicalpremium.csv')

data.head()

,Age,Diabetes,BloodPressureProblems,AnyTransplants,AnyChronicDiseases,Height,Weight,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries,PremiumPrice
0,45,0,0,0,0,155,57,0,0,0,25000
1,60,1,0,0,0,180,73,0,0,0,29000
2,36,1,1,0,0,158,59,0,0,1,23000
3,52,1,1,0,1,183,93,0,0,2,28000
4,38,0,0,0,1,166,88,0,0,1,23000


In [5]:
#Preprocess data
X = data.drop('PremiumPrice', axis=1)
num_columns = X.shape[1]
print("num of features: " + str(num_columns))
y = data['PremiumPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled)


num of features: 10
[[-0.33780827 -0.83363909 -0.93575345 ...  1.94106722 -0.36528836
   0.47984854]
 [ 0.16463054 -0.83363909 -0.93575345 ... -0.51518051 -0.36528836
  -0.88929459]
 [ 1.74372396  1.19955988 -0.93575345 ... -0.51518051 -0.36528836
  -0.88929459]
 ...
 [ 0.23640752 -0.83363909  1.06865756 ... -0.51518051 -0.36528836
  -0.88929459]
 [-1.27090893 -0.83363909  1.06865756 ... -0.51518051 -0.36528836
   0.47984854]
 [ 1.16950817  1.19955988 -0.93575345 ... -0.51518051 -0.36528836
  -0.88929459]]


In [ ]:
#Hyperparameter tuning

#Hidden Layers
[1,2,3,4,5]

#Neurons in hidden layer
[3, 5, 7, 10]

#Optimizer
['adamW', 'sgd', 'Adadelta']

#Learning rate
[0.001, 0.01, 0.1]

#Batch size
[32, 64, 128]

#Epochs
[100, 200, 300]

#Regularization
[0.01, 0.1, 0.5]

#num of parameters: 5*4*3*3*3*3*3 = 9720

In [25]:
import tensorflow as tf
from sklearn.model_selection import ParameterSampler
import numpy as np

# Define your TensorFlow ANN model
def create_model(hidden_layers, neurons, optimizer, learning_rate, regularization):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(neurons, input_shape=(input_dim,)))
    model.add(tf.keras.layers.LeakyReLU())
    for _ in range(hidden_layers - 1):
        model.add(tf.keras.layers.Dense(neurons))
        model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    
    if optimizer == 'adamW':
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'Adadelta':
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', root_mean_squared_error, mean_absolute_percentage_error])
    return model

# Define the parameter grid for grid search
param_grid = {
    'hidden_layers': [1, 2, 3, 4, 5],
    'neurons': [3, 5, 7, 10],
    'optimizer': ['adamW', 'sgd', 'Adadelta'],
    'learning_rate': [0.001, 0.01, 0.1],
    'regularization': [0.01, 0.1, 0.5]
}

# Create a ParameterSampler
param_list = list(ParameterSampler(param_grid, n_iter=1, random_state=42))

# For each set of parameters
for params in param_list:
    # Create a model
    model = create_model(**params)
    
    # Train the model
    model.fit(X, y, epochs=100, batch_size=32, verbose=0)
    
    # Evaluate the model
    score = model.evaluate(X, y, verbose=0)
    
    # Print the score
    print("Parameters: ", params)
    print("Score: ", score)


NameError: name 'root_mean_squared_error' is not defined

In [18]:
import tensorflow as tf
from sklearn.model_selection import ParameterSampler
import numpy as np

# Assume X and y are defined somewhere above this code
input_dim = X.shape[1]

# Define your TensorFlow ANN model
def create_model(hidden_layers, neurons, optimizer, learning_rate, regularization):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(neurons, input_shape=(input_dim,)))
    model.add(tf.keras.layers.LeakyReLU())
    for _ in range(hidden_layers - 1):
        model.add(tf.keras.layers.Dense(neurons))
        model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    
    if optimizer == 'adamW':
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'Adadelta':
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError()])
    return model

# Define the parameter grid for grid search
param_grid = {
    'hidden_layers': [2, 3, 4, 5, 6,7],
    'neurons': [3, 5, 6, 7,8, 9, 10],
    'optimizer': ['adamW', 'sgd', 'Adadelta'],
    'learning_rate': [0.001, 0.01],
    'regularization': [0.01, 0.1]
}

# Create a ParameterSampler
param_list = list(ParameterSampler(param_grid, n_iter=500, random_state=42))
best_score = np.inf
best_scores = None
best_params = None
# For each set of parameters
for params in param_list:
    # Create a model
    model = create_model(**params)
    
    # Train the model
    model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, verbose=0)
    
    # Evaluate the model
    score = model.evaluate(X_test_scaled, y_test, verbose=0)
    
    #Print the score
    #print("Parameters: ", params)
    #print("Score: ", score)

    if score[0] < best_score:
        best_score = score[0]
        best_params = params
        best_scores = score

print("Best Parameters: ", best_params)
print("Best Score: ", best_score)
print(best_scores)


Best Parameters:  {'regularization': 0.1, 'optimizer': 'adamW', 'neurons': 9, 'learning_rate': 0.01, 'hidden_layers': 7}
Best Score:  11821616.0
[11821616.0, 2333.752197265625, 3438.2578125, 9.844259262084961]


In [10]:
print("Best scores: ", best_scores)

Best scores:  [12293031.0, 2383.6201171875, 3506.141845703125, 10.432963371276855]


In [15]:
#hidden_layers, neurons, optimizer, learning_rate, regularization    
model = create_model(4, 10, 'adamW', 0.01, 0.01)


# Train the model
model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, verbose = 0)

# Evaluate the model
score = model.evaluate(X_test_scaled, y_test, verbose=0)

print(score)

[15617910.0, 2858.48876953125, 3951.9501953125, 12.902116775512695]
